# 病床姿勢監測 - 資料準備與對齊（Version 4 - 修正累積誤差）

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-06  
**版本**: v4 - 修正 Thermal 時間戳累積誤差問題

## 修正重點
- ✅ **使用全域幀計數器**，避免秒級分組的累積誤差
- ✅ **微秒計算修正**：直接計算微秒而非毫秒*1000
- ✅ **時間戳不再被修改**：保持原始配對時間的一致性
- ✅ **簡化配對邏輯**：直接按時間戳配對，移除複雜的 burst/dropout 處理

## 功能
1. 讀取 output.txt 取得相機開始時間
2. 載入熱成像資料（全域幀計數器，無累積誤差）
3. 分析 RGB 影片資訊
4. **直接時間戳配對**（簡單且準確）
5. 翻轉檢測與修復
6. 匯出 RGB 圖片供 LabelMe 標註
7. 儲存對齊後的 Thermal 資料

## 輸出
- `output/labelme_project/` - LabelMe 標註專案
- `output/aligned_dataset/` - 對齊後的訓練資料
- `output/pairing_analysis/` - 配對分析結果

---
## Step 0: 環境設定與套件匯入

In [1]:
import os
import sys
import json
import re
import cv2
import numpy as np
import multiprocessing as mp
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # 使用非互動式後端（適用於 SSH 環境）
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')

print("="*70)
print("病床姿勢監測系統 - 資料準備模組 v4（修正累積誤差）")
print("="*70)
print(f"執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print("="*70)

病床姿勢監測系統 - 資料準備模組 v4（修正累積誤差）
執行時間: 2025-11-08 02:25:11
使用者: rochi190
Python 版本: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
NumPy 版本: 2.3.4
OpenCV 版本: 4.10.0


---
## Step 1: 路徑與參數配置

In [2]:
# ========================================
# 路徑配置
# ========================================
NUM_WORKERS = max(1, mp.cpu_count() - 1)
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output_v7'
PAIRING_DIR = OUTPUT_DIR / 'pairing_analysis'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
DIAGNOSTIC_DIR = OUTPUT_DIR / 'diagnostics'

for directory in [PAIRING_DIR, LABELME_DIR, ALIGNED_DIR, DIAGNOSTIC_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# ========================================
# 熱成像參數
# ========================================
THERMAL_RESOLUTION = (32, 24)  # (寬度, 高度)
THERMAL_WIDTH, THERMAL_HEIGHT = THERMAL_RESOLUTION

# ========================================
# 配對參數
# ========================================
TARGET_FPS = 8  # 目標 FPS
FRAME_INTERVAL_MS = 1000 / TARGET_FPS  # 125ms

# 翻轉設定（執行 Step 7 後調整）
FLIP_THERMAL = True
FLIP_RGB = False

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  Thermal 目錄: {'✓' if THERMAL_DIR.exists() else '✗'} {THERMAL_DIR}")
print(f"  Camera 目錄: {'✓' if CAMERA_DIR.exists() else '✗'} {CAMERA_DIR}")
print(f"  output.txt: {'✓' if OUTPUT_TXT.exists() else '✗'} {OUTPUT_TXT}")
print(f"  AVI 檔案: {'✓' if AVI_FILE.exists() else '✗'} {AVI_FILE}")

print(f"\n⚙️ 配對參數:")
print(f"  目標 FPS: {TARGET_FPS}")
print(f"  幀間隔: {FRAME_INTERVAL_MS:.1f}ms")

print(f"\n🔄 翻轉設定:")
print(f"  翻轉 Thermal: {FLIP_THERMAL}")
print(f"  翻轉 RGB: {FLIP_RGB}")


📁 路徑檢查:
  Thermal 目錄: ✓ /home/gary/claude4.5/data/thermal
  Camera 目錄: ✓ /home/gary/claude4.5/data/camera
  output.txt: ✓ /home/gary/claude4.5/data/camera/output.txt
  AVI 檔案: ✓ /home/gary/claude4.5/data/camera/output.avi

⚙️ 配對參數:
  目標 FPS: 8
  幀間隔: 125.0ms

🔄 翻轉設定:
  翻轉 Thermal: True
  翻轉 RGB: False


---
## Step 2: 解析 output.txt（相機開始時間）

In [3]:
def parse_output_txt(txt_path: Path) -> Optional[datetime]:
    """
    解析 output.txt 取得相機開始時間
    """
    if not txt_path.exists():
        print(f"❌ {txt_path} 不存在")
        return None
    
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        with open(txt_path, 'r', encoding='big5') as f:
            content = f.read()
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"檔案大小: {len(content)} bytes")
    print(f"\n完整內容:\n{content}")
    print(f"{'='*70}")
    
    # 時間格式模式
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
    ]
    
    for pattern, fmt in patterns:
        match = re.search(pattern, content)
        if match:
            try:
                camera_start = datetime.strptime(match.group(1), fmt)
                print(f"\n✅ 找到相機開始時間: {camera_start}")
                return camera_start
            except:
                continue
    
    print(f"\n❌ 無法解析時間")
    return None

# 執行解析
camera_start_time = parse_output_txt(OUTPUT_TXT)

if camera_start_time is None:
    raise RuntimeError("❌ 無法解析 output.txt，請檢查檔案格式")

print(f"\n📌 相機開始時間: {camera_start_time}")


解析 output.txt
檔案大小: 898 bytes

完整內容:
viedo_start_2020-09-24 13:09:30
lay_mid_2020-09-24 13:11:08 
end_time_2020-09-24 13:11:43 
lay_right_2020-09-24 13:12:02 
end_time_2020-09-24 13:12:39 
lay_left_2020-09-24 13:12:58 
end_time_2020-09-24 13:13:36 
bed_2020-09-24 13:13:53 
end_time_2020-09-24 13:14:32 
sit_right_2020-09-24 13:14:40 
end_time_2020-09-24 13:15:22 
sit_left_2020-09-24 13:15:27 
end_time_2020-09-24 13:16:04 
Falldown_2020-09-24 13:16:38 
end_time_2020-09-24 13:16:43 
Falldown_2020-09-24 13:16:57 
end_time_2020-09-24 13:17:01 
Falldown_2020-09-24 13:17:38 
end_time_2020-09-24 13:17:42 
Falldown_2020-09-24 13:17:53 
end_time_2020-09-24 13:17:58 
Falldown_2020-09-24 13:18:09 
end_time_2020-09-24 13:18:12 
Falldown_2020-09-24 13:18:34 
end_time_2020-09-24 13:18:38 
Falldown_2020-09-24 13:19:06 
end_time_2020-09-24 13:19:08 
Falldown_2020-09-24 13:20:07 
Falldown_2020-09-24 13:20:09 
Falldown_2020-09-24 13:20:18 


✅ 找到相機開始時間: 2020-09-24 13:09:30

📌 相機開始時間: 2020-09-24 13:09:30

---
## Step 3: 載入熱成像資料（全域計數器版本 - 修正累積誤差）

In [4]:
def load_thermal_data_with_global_counter(log_files: List[Path]) -> List[Dict]:
    """
    載入熱成像資料（使用全域幀計數器，避免累積誤差）
    
    修正重點：
    1. 使用全域幀計數器，不依賴秒級分組
    2. 從第一幀開始，每幀間隔固定 125ms
    3. 避免 burst/dropout 導致的時間跳躍
    """
    print(f"\n{'='*70}")
    print(f"載入熱成像資料（全域計數器版本）")
    print(f"{'='*70}")
    print(f"找到 {len(log_files)} 個 log 檔案")
    print(f"🔧 使用全域幀計數器，每幀間隔 {FRAME_INTERVAL_MS:.1f}ms")
    
    all_data = []
    global_frame_count = 0
    first_timestamp = None
    
    for log_file in tqdm(log_files, desc="載入 Thermal"):
        try:
            with open(log_file, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    
                    try:
                        data_dict = json.loads(line)
                        time_parts = data_dict.get('time', [])
                        
                        if len(time_parts) < 6:
                            continue
                        
                        year, month, day, hour, minute, second = time_parts[:6]
                        original_timestamp = datetime(year, month, day, hour, minute, second)
                        
                        # 記錄第一幀的時間戳作為基準
                        if first_timestamp is None:
                            first_timestamp = original_timestamp
                        
                        # 🔧 使用全域計數器計算精確時間戳
                        offset_ms = global_frame_count * FRAME_INTERVAL_MS
                        timestamp = first_timestamp + timedelta(milliseconds=offset_ms)
                        
                        message = data_dict.get('message', [])
                        if len(message) != THERMAL_WIDTH * THERMAL_HEIGHT:
                            continue
                        
                        thermal_image = np.array(message, dtype=np.uint8).reshape(
                            THERMAL_HEIGHT, THERMAL_WIDTH
                        )
                        
                        all_data.append({
                            'timestamp': timestamp,
                            'image': thermal_image,
                            'frame_number': global_frame_count,
                            'original_timestamp': original_timestamp
                        })
                        
                        global_frame_count += 1
                        
                    except Exception as e:
                        continue
        
        except Exception as e:
            print(f"❌ 載入失敗 {log_file.name}: {e}")
            continue
    
    print(f"\n✅ 載入完成")
    print(f"  總幀數: {len(all_data):,}")
    
    if all_data:
        thermal_start = all_data[0]['timestamp']
        thermal_end = all_data[-1]['timestamp']
        duration = (thermal_end - thermal_start).total_seconds()
        
        print(f"  第一幀: {thermal_start}")
        print(f"  最後幀: {thermal_end}")
        print(f"  時長: {duration:.2f} 秒")
        print(f"  平均 FPS: {len(all_data) / duration:.2f}")
        
        # 驗證時間間隔
        time_diffs = []
        for i in range(min(100, len(all_data) - 1)):
            dt = (all_data[i+1]['timestamp'] - all_data[i]['timestamp']).total_seconds() * 1000
            time_diffs.append(dt)
        
        print(f"\n  時間間隔統計（前 100 幀）:")
        print(f"    平均: {np.mean(time_diffs):.1f}ms")
        print(f"    標準差: {np.std(time_diffs):.1f}ms")
        print(f"    最小值: {min(time_diffs):.1f}ms")
        print(f"    最大值: {max(time_diffs):.1f}ms")
        print(f"    預期: {FRAME_INTERVAL_MS:.1f}ms")
    
    print("="*70)
    
    return all_data

# 載入所有 thermal log
log_files = sorted(THERMAL_DIR.glob('log*.txt'))

if not log_files:
    raise FileNotFoundError(f"❌ 未找到 log 檔案於 {THERMAL_DIR}")

therm_data = load_thermal_data_with_global_counter(log_files)


載入熱成像資料（全域計數器版本）
找到 5 個 log 檔案
🔧 使用全域幀計數器，每幀間隔 125.0ms


載入 Thermal: 100%|██████████| 5/5 [00:00<00:00, 11.18it/s]


✅ 載入完成
  總幀數: 5,318
  第一幀: 2020-09-24 13:09:30
  最後幀: 2020-09-24 13:20:34.625000
  時長: 664.62 秒
  平均 FPS: 8.00

  時間間隔統計（前 100 幀）:
    平均: 125.0ms
    標準差: 0.0ms
    最小值: 125.0ms
    最大值: 125.0ms
    預期: 125.0ms


---
## Step 4: 分析 RGB 影片

In [ ]:
# 分析 AVI 影片
cap = cv2.VideoCapture(str(AVI_FILE))

if not cap.isOpened():
    raise RuntimeError(f"❌ 無法開啟影片 {AVI_FILE}")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps if fps > 0 else 0

cap.release()

print(f"\n{'='*70}")
print(f"RGB 影片資訊")
print(f"{'='*70}")
print(f"  FPS: {fps:.2f}")
print(f"  總幀數: {frame_count:,}")
print(f"  解析度: {width} × {height}")
print(f"  時長: {duration:.2f} 秒")
print(f"  開始時間: {camera_start_time}")
print(f"  結束時間: {camera_start_time + timedelta(seconds=duration)}")
print(f"\n  預期 Thermal-RGB 幀間隔:")
print(f"    Thermal 間隔: {FRAME_INTERVAL_MS:.1f}ms")
print(f"    RGB 間隔: {1000/fps:.1f}ms")
print(f"    每個 Thermal 對應 RGB 幀數: {FRAME_INTERVAL_MS / (1000/fps):.1f}")
print("="*70)

---
## Step 5: 簡化配對（直接按時間戳配對）

In [ ]:
def direct_timestamp_pairing(
    thermal_data: List[Dict],
    camera_start_time: datetime,
    fps: float,
    frame_count: int
) -> List[Dict]:
    """
    直接按時間戳配對 Thermal 和 RGB（簡化版）
    
    優點：
    1. 不修改時間戳
    2. 邏輯簡單清晰
    3. 沒有累積誤差
    """
    print(f"\n{'='*70}")
    print(f"直接時間戳配對")
    print(f"{'='*70}")
    
    pairs = []
    rgb_end_time = camera_start_time + timedelta(seconds=frame_count / fps)
    
    print(f"\n配對範圍:")
    print(f"  Camera Start: {camera_start_time}")
    print(f"  Camera End: {rgb_end_time}")
    
    skipped_before = 0
    skipped_after = 0
    
    for thermal_frame in tqdm(thermal_data, desc="配對 Thermal-RGB"):
        thermal_ts = thermal_frame['timestamp']
        
        # 檢查是否在 RGB 影片時間範圍內
        if thermal_ts < camera_start_time:
            skipped_before += 1
            continue
        
        if thermal_ts > rgb_end_time:
            skipped_after += 1
            continue
        
        # 計算對應的 RGB 幀索引
        offset = (thermal_ts - camera_start_time).total_seconds()
        rgb_frame_idx = int(round(offset * fps))
        rgb_frame_idx = max(0, min(rgb_frame_idx, frame_count - 1))
        
        # 計算配對誤差
        actual_time = rgb_frame_idx / fps
        rgb_error = abs(actual_time - offset)
        
        pairs.append({
            'pair_id': f'pair_{len(pairs):05d}',
            'thermal': thermal_frame,
            'rgb_frame_idx': rgb_frame_idx,
            'timestamp': thermal_ts,
            'frame_number': thermal_frame['frame_number'],
            'rgb_error_ms': rgb_error * 1000
        })
    
    print(f"\n配對結果:")
    print(f"  總配對數: {len(pairs):,}")
    print(f"  跳過（太早）: {skipped_before}")
    print(f"  跳過（太晚）: {skipped_after}")
    print(f"  成功率: {len(pairs) / len(thermal_data) * 100:.1f}%")
    
    if pairs:
        rgb_errors = [p['rgb_error_ms'] for p in pairs]
        print(f"\n  RGB 配對誤差:")
        print(f"    平均: {np.mean(rgb_errors):.2f}ms")
        print(f"    中位數: {np.median(rgb_errors):.2f}ms")
        print(f"    最大值: {max(rgb_errors):.2f}ms")
        
        # RGB 幀間隔分佈
        rgb_indices = [p['rgb_frame_idx'] for p in pairs]
        intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
        interval_counts = Counter(intervals)
        
        print(f"\n  RGB 幀間隔分佈:")
        for interval in sorted(interval_counts.keys())[:10]:
            count = interval_counts[interval]
            percentage = count / len(intervals) * 100
            print(f"    間隔 {interval:3d} 幀: {count:5d} 次 ({percentage:5.1f}%)")
    
    print("="*70)
    
    return pairs

# 執行配對
pairs_adaptive = direct_timestamp_pairing(
    therm_data, camera_start_time, fps, frame_count
)

print(f"\n📌 配對結果: {len(pairs_adaptive):,} 對")
# ========================================
# Step 5.5: 修復後面大 Dropout
# ========================================

def analyze_and_fix_all_dropouts(thermal_data, camera_start_time, fps, frame_count):
    """
    通用 dropout 修復：不只修後面，把所有 dropout 都修好
    """
    
    print(f"\n{'='*70}")
    print(f"全面分析和修復 Dropout（通用版）")
    print(f"{'='*70}")
    
    # Step 1: 分析 Thermal 時間分佈
    sections = {}
    for frame in thermal_data:
        offset = (frame['timestamp'] - camera_start_time).total_seconds()
        if offset < 0:
            continue
        second = int(offset)
        if second not in sections:
            sections[second] = []
        sections[second].append(frame)
    
    if not sections:
        print("❌ 無資料")
        return []
    
    min_sec = min(sections.keys())
    max_sec = max(sections.keys())
    
    print(f"\n1️⃣ 時間分佈分析:")
    print(f"   時間範圍: 第 {min_sec} ~ {max_sec} 秒")
    
    # 統計每秒的幀數
    sec_counts = {sec: len(frames) for sec, frames in sections.items()}
    
    print(f"   正常秒 (8幀): {sum(1 for c in sec_counts.values() if c == 8)} 秒")
    print(f"   少幀秒: {sum(1 for c in sec_counts.values() if 0 < c < 8)} 秒")
    print(f"   多幀秒: {sum(1 for c in sec_counts.values() if c > 8)} 秒")
    print(f"   缺失秒: {max_sec - min_sec + 1 - len(sections)} 秒")
    
    # Step 2: 為缺失的秒建立空 section
    for sec in range(min_sec, max_sec + 1):
        if sec not in sections:
            sections[sec] = []
    
    # Step 3: 分類
    burst_secs = {}  # 多幀
    dropout_secs = []  # 少幀或無幀
    
    for sec in range(min_sec, max_sec + 1):
        count = len(sections[sec])
        if count == 0 or count < 8:
            dropout_secs.append(sec)
        elif count > 8:
            burst_secs[sec] = count
    
    print(f"\n2️⃣ 問題秒分類:")
    print(f"   Burst 秒: {len(burst_secs)} 秒")
    print(f"   Dropout 秒: {len(dropout_secs)} 秒")
    
    # Step 4: 智能分配
    print(f"\n3️⃣ 幀重新分配:")
    
    total_redistributed = 0
    
    # 策略：從 burst 秒取多餘的幀，分配給前面或周圍的 dropout 秒
    for burst_sec in sorted(burst_secs.keys()):
        frames = sections[burst_sec]
        excess_count = len(frames) - 8
        
        # 找附近的 dropout（優先前面，再看後面）
        nearby_dropouts = []
        
        # 往前找（在 burst 秒前面）
        for ds in sorted(dropout_secs, reverse=True):
            if ds < burst_sec:
                needed = 8 - len(sections[ds])
                if needed > 0:
                    nearby_dropouts.append((ds, needed))
        
        if not nearby_dropouts:
            # 往後找
            for ds in sorted(dropout_secs):
                if ds > burst_sec:
                    needed = 8 - len(sections[ds])
                    if needed > 0:
                        nearby_dropouts.append((ds, needed))
        
        if not nearby_dropouts:
            print(f"   第 {burst_sec} 秒: {excess_count} 幀多餘，無可分配（丟棄）")
            sections[burst_sec] = frames[-8:]
            continue
        
        # 分配
        excess_frames = frames[:excess_count]
        sections[burst_sec] = frames[excess_count:]
        
        frame_idx = 0
        for dropout_sec, needed in nearby_dropouts:
            if frame_idx >= len(excess_frames):
                break
            
            available = min(needed, len(excess_frames) - frame_idx)
            
            for i in range(available):
                frame = excess_frames[frame_idx]
                
                # 修正時間戳
                frame_in_sec = len(sections[dropout_sec])
                new_ms = int(frame_in_sec * FRAME_INTERVAL_MS)
                new_ts = camera_start_time + timedelta(seconds=dropout_sec, milliseconds=new_ms)
                frame['timestamp'] = new_ts
                
                sections[dropout_sec].append(frame)
                frame_idx += 1
            
            print(f"   第 {burst_sec} 秒 → 第 {dropout_sec} 秒: 補 {available} 幀")
        
        total_redistributed += frame_idx
    
    print(f"   總重新分配: {total_redistributed} 幀")
    
    # Step 5: 配對
    print(f"\n4️⃣ 最終配對:")
    
    pairs = []
    rgb_end_time = camera_start_time + timedelta(seconds=frame_count / fps)
    
    for sec in sorted(sections.keys()):
        for thermal_frame in sections[sec]:
            thermal_ts = thermal_frame['timestamp']
            
            if thermal_ts < camera_start_time or thermal_ts > rgb_end_time:
                continue
            
            offset = (thermal_ts - camera_start_time).total_seconds()
            rgb_frame_idx = int(round(offset * fps))
            rgb_frame_idx = max(0, min(rgb_frame_idx, frame_count - 1))
            
            actual_time = rgb_frame_idx / fps
            rgb_error = abs(actual_time - offset)
            
            pairs.append({
                'pair_id': f'pair_{len(pairs):05d}',
                'thermal': thermal_frame,
                'rgb_frame_idx': rgb_frame_idx,
                'timestamp': thermal_ts,
                'frame_number': thermal_frame['frame_number'],
                'rgb_error_ms': rgb_error * 1000
            })
    
    print(f"   總配對數: {len(pairs):,}")
    print("="*70)
    
    return pairs

# 直接用這個取代 Step 5
pairs_adaptive = analyze_and_fix_all_dropouts(therm_data, camera_start_time, fps, frame_count)

---
## Step 6: 時間同步診斷

In [ ]:
def diagnose_timing_sync(pairs: List[Dict], camera_start_time: datetime, fps: float, output_dir: Path):
    """
    診斷時間同步問題
    """
    print(f"\n{'='*70}")
    print(f"時間同步診斷")
    print(f"{'='*70}")
    
    # 計算每個配對的時間偏移
    time_offsets = []
    
    for pair in pairs[:min(500, len(pairs))]:
        thermal_ts = pair['timestamp']
        rgb_frame_idx = pair['rgb_frame_idx']
        
        # RGB 的實際時間
        rgb_actual_time = camera_start_time + timedelta(seconds=rgb_frame_idx / fps)
        
        # 時間差（正值 = Thermal 慢於 RGB）
        time_diff = (thermal_ts - rgb_actual_time).total_seconds()
        time_offsets.append(time_diff)
    
    mean_offset = np.mean(time_offsets)
    std_offset = np.std(time_offsets)
    
    print(f"\n時間偏移統計（前 {len(time_offsets)} 個配對）:")
    print(f"  平均: {mean_offset:.3f} 秒")
    print(f"  標準差: {std_offset:.3f} 秒")
    print(f"  最小值: {min(time_offsets):.3f} 秒")
    print(f"  最大值: {max(time_offsets):.3f} 秒")
    
    # 視覺化
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # 時間差分布
    axes[0].hist(time_offsets, bins=50, edgecolor='black', alpha=0.7, color='#ff6b6b')
    axes[0].axvline(mean_offset, color='red', linestyle='--', linewidth=2,
                    label=f'Mean: {mean_offset:.3f}s')
    axes[0].axvline(0, color='green', linestyle='-', linewidth=1, alpha=0.5,
                    label='Perfect Sync')
    axes[0].set_xlabel('Time Offset (seconds, positive = Thermal slower)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Thermal-RGB Time Offset Distribution')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 時間差趨勢
    axes[1].plot(range(len(time_offsets)), time_offsets, 'b-', linewidth=1, alpha=0.7)
    axes[1].axhline(mean_offset, color='red', linestyle='--', linewidth=2,
                    label=f'Mean: {mean_offset:.3f}s')
    axes[1].axhline(0, color='green', linestyle='-', linewidth=1, alpha=0.5,
                    label='Perfect Sync')
    axes[1].set_xlabel('Pair Index')
    axes[1].set_ylabel('Time Offset (seconds)')
    axes[1].set_title('Time Offset Trend Over Time')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = output_dir / 'timing_diagnosis_v4.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ 診斷圖已儲存: {save_path}")
    
    if abs(mean_offset) > 0.5:
        print(f"\n⚠️ 警告：時間偏移超過 0.5 秒！")
        print(f"   建議修正 camera_start_time:")
        corrected = camera_start_time + timedelta(seconds=mean_offset)
        print(f"   camera_start_time = datetime({corrected.year}, {corrected.month}, {corrected.day}, ")
        print(f"                                {corrected.hour}, {corrected.minute}, {corrected.second})")
    else:
        print(f"\n✅ 時間同步良好（偏移 < 0.5 秒）")
    
    print("="*70)

# 執行診斷
diagnose_timing_sync(pairs_adaptive, camera_start_time, fps, DIAGNOSTIC_DIR)

In [ ]:
def detailed_diagnostics(pairs: List[Dict], camera_start_time: datetime, fps: float) -> Dict:
    """
    詳細診斷：找出 RGB FPS 和 Thermal 時間戳的真實情況
    """
    
    print(f"\n{'='*70}")
    print(f"詳細診斷：找出真正的延遲原因")
    print(f"{'='*70}")
    
    if len(pairs) < 100:
        print("配對數量太少，無法診斷")
        return {}
    
    # ========== 診斷 1: RGB 幀序列的真實 FPS ==========
    print(f"\n【診斷 1】RGB 幀序列分析:")
    
    rgb_indices = [p['rgb_frame_idx'] for p in pairs]
    
    # 計算每 100 個配對的 RGB 幀消耗
    intervals_100 = []
    for i in range(0, len(rgb_indices) - 100, 100):
        rgb_consumed = rgb_indices[i+100] - rgb_indices[i]
        intervals_100.append(rgb_consumed)
    
    if intervals_100:
        print(f"  每 100 個 Thermal 幀消耗的 RGB 幀數:")
        print(f"    平均: {np.mean(intervals_100):.2f} 幀")
        print(f"    最小: {np.min(intervals_100)} 幀")
        print(f"    最大: {np.max(intervals_100)} 幀")
        print(f"    標準差: {np.std(intervals_100):.2f} 幀")
        print(f"    預期: 308.375 幀 (100 * 24.67 / 8)")
        
        # 檢查是否呈線性趨勢
        if len(intervals_100) > 1:
            trend = intervals_100[-1] - intervals_100[0]
            if abs(trend) > 2:
                print(f"  ⚠️ 檢測到趨勢: {trend:+.2f} 幀 (從開始到結束)")
                print(f"    意味著 RGB 幀消耗**在加速或減速**")
            else:
                print(f"  ✓ 沒有明顯趨勢")
    
    # ========== 診斷 2: Thermal 時間戳均勻性 ==========
    print(f"\n【診斷 2】Thermal 時間戳均勻性:")
    
    thermal_times = []
    for i, pair in enumerate(pairs):
        offset = (pair['timestamp'] - camera_start_time).total_seconds()
        thermal_times.append(offset)
    
    thermal_times = np.array(thermal_times)
    
    # 計算相鄰幀的時間差
    time_intervals = np.diff(thermal_times)
    
    print(f"  Thermal 幀時間間隔:")
    print(f"    平均: {np.mean(time_intervals)*1000:.2f} ms")
    print(f"    最小: {np.min(time_intervals)*1000:.2f} ms")
    print(f"    最大: {np.max(time_intervals)*1000:.2f} ms")
    print(f"    標準差: {np.std(time_intervals)*1000:.2f} ms")
    print(f"    預期: 125.00 ms")
    
    # 檢查是否有異常間隔
    abnormal_threshold = 125 + 10  # 允許 ±10ms 偏差
    abnormal_count = np.sum(time_intervals*1000 > abnormal_threshold)
    
    if abnormal_count > 0:
        print(f"  ⚠️ 檢測到 {abnormal_count} 個異常間隔 (> {abnormal_threshold}ms)")
        
        # 找出異常的幀
        abnormal_indices = np.where(time_intervals*1000 > abnormal_threshold)[0]
        print(f"  異常間隔位置 (前 10 個):")
        for idx in abnormal_indices[:10]:
            interval_ms = time_intervals[idx] * 1000
            print(f"    幀 {idx} → {idx+1}: {interval_ms:.2f} ms")
    else:
        print(f"  ✓ 所有時間間隔正常")
    
    # ========== 診斷 3: 配對誤差的累積趨勢 ==========
    print(f"\n【診斷 3】配對誤差累積趨勢:")
    
    # 計算每個配對的「理想 RGB 位置」vs「實際 RGB 位置」
    errors = []
    
    for pair in pairs:
        thermal_offset = (pair['timestamp'] - camera_start_time).total_seconds()
        
        # 理想的 RGB 幀索引（如果 RGB FPS 完全是 25）
        ideal_rgb_idx = thermal_offset * fps
        
        # 實際的 RGB 幀索引
        actual_rgb_idx = pair['rgb_frame_idx']
        
        # 誤差（幀數）
        error = actual_rgb_idx - ideal_rgb_idx
        errors.append(error)
    
    errors = np.array(errors)
    
    print(f"  配對誤差（實際 - 理想）:")
    print(f"    平均: {np.mean(errors):+.4f} 幀")
    print(f"    最小: {np.min(errors):+.4f} 幀")
    print(f"    最大: {np.max(errors):+.4f} 幀")
    print(f"    標準差: {np.std(errors):.4f} 幀")
    
    # 檢查是否有線性趨勢
    if len(errors) > 1:
        x = np.arange(len(errors))
        coeffs = np.polyfit(x, errors, 1)
        slope = coeffs[0]
        
        print(f"\n  誤差線性趨勢:")
        print(f"    斜率: {slope:.6f} 幀/對")
        print(f"    預計 1000 幀後誤差: {slope * 1000:+.2f} 幀")
        
        if abs(slope) > 0.0001:
            print(f"  ⚠️ 檢測到明顯的單向誤差累積")
            direction = "越來越快" if slope < 0 else "越來越慢"
            print(f"    Thermal 配對 {direction}")
        else:
            print(f"  ✓ 誤差基本穩定")
    
    # ========== 診斷 4: RGB FPS 實際值 ==========
    print(f"\n【診斷 4】實際 RGB FPS 計算:")
    
    # 用總時間和總幀數推算
    total_thermal_time = thermal_times[-1] - thermal_times[0]
    total_rgb_frames = rgb_indices[-1] - rgb_indices[0]
    
    actual_fps = total_rgb_frames / total_thermal_time
    
    print(f"  基於配對數據反推:")
    print(f"    總 Thermal 時長: {total_thermal_time:.2f} 秒")
    print(f"    總 RGB 幀消耗: {total_rgb_frames} 幀")
    print(f"    實際 RGB FPS: {actual_fps:.4f}")
    print(f"    設定 RGB FPS: {fps:.4f}")
    print(f"    差異: {actual_fps - fps:+.4f}")
    
    if abs(actual_fps - fps) > 0.1:
        print(f"  ⚠️ 實際 FPS 與設定 FPS 差異大")
        print(f"    這可能是導致延遲的主要原因")
    else:
        print(f"  ✓ FPS 基本匹配")
    
    # ========== 診斷 5: 視覺化 ==========
    print(f"\n【診斷 5】生成診斷圖表...")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 子圖 1: RGB 幀消耗趨勢
    sample_indices = np.linspace(0, len(rgb_indices)-1, 100, dtype=int)
    rgb_at_samples = [rgb_indices[i] for i in sample_indices]
    thermal_at_samples = sample_indices.tolist()
    
    axes[0, 0].plot(thermal_at_samples, rgb_at_samples, 'b-', linewidth=2, label='Actual RGB frames')
    
    # 理想線（FPS = 24.67）
    ideal_rgb = [idx * fps / 8 for idx in thermal_at_samples]
    axes[0, 0].plot(thermal_at_samples, ideal_rgb, 'r--', linewidth=2, label=f'Ideal (FPS={fps:.2f})')
    
    axes[0, 0].set_xlabel('Thermal Frame Index')
    axes[0, 0].set_ylabel('RGB Frame Index')
    axes[0, 0].set_title('RGB Frame Progression')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 子圖 2: 時間間隔分佈
    axes[0, 1].hist(time_intervals*1000, bins=50, edgecolor='black', alpha=0.7, color='#4ecdc4')
    axes[0, 1].axvline(125, color='red', linestyle='--', linewidth=2, label='Expected: 125ms')
    axes[0, 1].set_xlabel('Time Interval (ms)')
    axes[0, 1].set_ylabel('Count')
    axes[0, 1].set_title('Thermal Frame Time Interval Distribution')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 子圖 3: 配對誤差趨勢
    axes[1, 0].scatter(range(len(errors)), errors, alpha=0.3, s=5, color='#95e1d3')
    if len(errors) > 1:
        z = np.polyfit(range(len(errors)), errors, 1)
        p = np.poly1d(z)
        axes[1, 0].plot(range(len(errors)), p(range(len(errors))), 'r-', linewidth=2, label=f'Trend: {z[0]:.6f}x')
        axes[1, 0].legend()
    
    axes[1, 0].set_xlabel('Pair Index')
    axes[1, 0].set_ylabel('Error (Frames)')
    axes[1, 0].set_title('Pairing Error Trend')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 子圖 4: 每 100 對的 RGB 消耗
    axes[1, 1].plot(intervals_100 if intervals_100 else [0], 'go-', linewidth=2, markersize=6)
    axes[1, 1].axhline(308.375, color='red', linestyle='--', linewidth=2, label='Expected: 308.375')
    axes[1, 1].set_xlabel('Sample Index (per 100 pairs)')
    axes[1, 1].set_ylabel('RGB Frames Consumed')
    axes[1, 1].set_title('RGB Frame Consumption per 100 Thermal Frames')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle('Detailed Synchronization Diagnostics', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    save_path = DIAGNOSTIC_DIR / 'detailed_diagnostics.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ 診斷圖已儲存: {save_path}")
    
    print("="*70)
    
    return {
        'thermal_interval_mean_ms': np.mean(time_intervals) * 1000,
        'thermal_interval_std_ms': np.std(time_intervals) * 1000,
        'error_slope': coeffs[0] if len(errors) > 1 else 0,
        'actual_fps': actual_fps,
        'expected_fps': fps
    }

# 執行詳細診斷
diagnostics = detailed_diagnostics(pairs_adaptive, camera_start_time, fps)

step6.5

In [ ]:
# ========================================
# Step 6.5: 時鐘漂移校正
# ========================================

def apply_clock_drift_correction(pairs, camera_start_time, fps, frame_count, drift_rate_ms_per_second):
    """
    應用時鐘漂移校正
    根據測得的延遲增長率，反向校正 Thermal 時間戳
    """
    print(f"\n{'='*70}")
    print(f"應用時鐘漂移校正")
    print(f"{'='*70}")
    print(f"檢測到的漂移率: {drift_rate_ms_per_second:.3f} ms/s")
    print(f"Thermal 比 RGB 快 {drift_rate_ms_per_second/10:.3f}%")
    
    corrected_pairs = []
    
    for i, pair in enumerate(pairs):
        pair_new = pair.copy()
        
        # 計算這一幀到開始的時間差（秒）
        elapsed_seconds = (pair['timestamp'] - pairs[0]['timestamp']).total_seconds()
        
        # 計算累積的漂移（毫秒）
        drift_accumulated_ms = elapsed_seconds * drift_rate_ms_per_second
        
        # 校正：把 Thermal 往後推（因為它比較快）
        corrected_ts = pair['timestamp'] - timedelta(milliseconds=drift_accumulated_ms)
        
        # 重新計算 RGB 幀索引
        offset_sec = (corrected_ts - camera_start_time).total_seconds()
        rgb_frame_idx = int(round(offset_sec * fps))
        rgb_frame_idx = max(0, min(rgb_frame_idx, frame_count - 1))
        
        pair_new['timestamp_original'] = pair['timestamp']
        pair_new['timestamp_corrected'] = corrected_ts
        pair_new['drift_correction_ms'] = drift_accumulated_ms
        pair_new['rgb_frame_idx'] = rgb_frame_idx
        
        corrected_pairs.append(pair_new)
    
    print(f"\n✅ 校正完成")
    print(f"  第一幀校正: 0.00 ms")
    print(f"  中間幀校正: {(pairs[len(pairs)//2]['timestamp'] - pairs[0]['timestamp']).total_seconds() * drift_rate_ms_per_second:.2f} ms")
    print(f"  最後幀校正: {(pairs[-1]['timestamp'] - pairs[0]['timestamp']).total_seconds() * drift_rate_ms_per_second:.2f} ms")
    print("="*70)
    
    return corrected_pairs

# 根據你的測量數據
# 延遲: 從 0ms (0:00) → 2700ms (6:50)
# 時長: 410 秒
measured_drift_rate_ms_per_second = 2700 / 410  # 大約 6.59 ms/s

print(f"\n計算漂移率:")
print(f"  測量延遲: 2700 ms")
print(f"  錄製時長: 410 秒")
print(f"  漂移率: {measured_drift_rate_ms_per_second:.3f} ms/s")

# 應用校正
pairs_final = apply_clock_drift_correction(
    pairs_adaptive, 
    camera_start_time, 
    fps,
    frame_count,
    measured_drift_rate_ms_per_second
)

step6.8

In [ ]:
"""def optimize_pairing_with_accumulator(pairs: List[Dict], fps: float, frame_count: int, target_thermal_fps: int = 8) -> List[Dict]:
    #使用浮點累積計數器消除單向延遲
    
    #核心想法：
    #1. 第一幀作為基準點（保持不變）
    #2. 用浮點累積器追蹤精確的 RGB 位置（不是整數）
    #3. 每一幀增加精確的間隔值
    #4. 只在最後四捨五入，保持浮點精度
    #5. 結果：誤差被限制在 ±0.5 幀，不會單向累積
    
    print(f"\n{'='*70}")
    print(f"V5.1: 累積計數器優化（消除單向延遲）")
    print(f"{'='*70}")
    
    if len(pairs) == 0:
        return pairs
    
    # 預期的 RGB 幀間隔（浮點數）
    expected_interval = fps / target_thermal_fps
    
    print(f"\n配置:")
    print(f"  Thermal FPS: {target_thermal_fps}")
    print(f"  RGB FPS: {fps:.2f}")
    print(f"  預期 RGB 間隔: {expected_interval:.5f} 幀")
    
    optimized_pairs = []
    
    # 第一幀保持原樣（作為基準）
    first_pair = pairs[0].copy()
    first_pair['accumulated_rgb_position'] = float(first_pair['rgb_frame_idx'])
    first_pair['rounding_error_ms'] = 0.0
    optimized_pairs.append(first_pair)
    
    # 浮點累積計數器（從第一幀開始）
    accumulated_rgb_position = float(pairs[0]['rgb_frame_idx'])
    
    rounding_errors = [0.0]  # 追蹤每一幀的四捨五入誤差
    interval_corrections = []
    
    print(f"\n開始優化配對...")
    
    for i in range(1, len(pairs)):
        # 增加精確的間隔（浮點數）
        accumulated_rgb_position += expected_interval
        
        # 四捨五入到最近的幀
        new_rgb_idx = int(round(accumulated_rgb_position))
        
        # 確保不超過邊界
        new_rgb_idx = max(0, min(new_rgb_idx, frame_count - 1))
        
        # 計算這一步的四捨五入誤差
        rounding_error = accumulated_rgb_position - new_rgb_idx
        rounding_errors.append(rounding_error)
        
        # 計算與上一幀的 RGB 間隔
        prev_rgb_idx = optimized_pairs[-1]['rgb_frame_idx']
        interval = new_rgb_idx - prev_rgb_idx
        interval_corrections.append(interval)
        
        # 建立優化後的配對
        pair_opt = pairs[i].copy()
        pair_opt['rgb_frame_idx_original'] = pairs[i]['rgb_frame_idx']
        pair_opt['rgb_frame_idx'] = new_rgb_idx
        pair_opt['accumulated_rgb_position'] = accumulated_rgb_position
        pair_opt['rounding_error'] = rounding_error
        pair_opt['rgb_interval_from_prev'] = interval
        
        optimized_pairs.append(pair_opt)
        
        # 每 1000 幀打印進度
        if (i + 1) % 1000 == 0:
            print(f"  已處理 {i+1:,} / {len(pairs):,} 對")
    
    print(f"\n✅ 優化完成")
    print(f"  總對數: {len(optimized_pairs):,}")
    
    # 詳細分析
    print(f"\n四捨五入誤差分析:")
    rounding_errors = np.array(rounding_errors)
    print(f"  平均: {np.mean(rounding_errors):+.4f} 幀")
    print(f"  最小: {np.min(rounding_errors):+.4f} 幀")
    print(f"  最大: {np.max(rounding_errors):+.4f} 幀")
    print(f"  標準差: {np.std(rounding_errors):.4f} 幀")
    
    # 關鍵指標：誤差是否被限制在 ±0.5
    out_of_range = np.sum(np.abs(rounding_errors) > 0.5)
    if out_of_range == 0:
        print(f"  ✅ 所有誤差都在 ±0.5 幀範圍內（完美）")
    else:
        print(f"  ⚠️ 有 {out_of_range} 個誤差超過 ±0.5 幀")
    
    # RGB 幀間隔分析
    print(f"\nRGB 幀間隔分析:")
    interval_corrections = np.array(interval_corrections)
    
    from collections import Counter
    interval_counter = Counter(interval_corrections)
    
    print(f"  最常見間隔: {sorted(interval_counter.items(), key=lambda x: x[1], reverse=True)[:5]}")
    print(f"  間隔平均: {np.mean(interval_corrections):.4f} 幀")
    print(f"  間隔標準差: {np.std(interval_corrections):.4f} 幀")
    print(f"  間隔最小值: {np.min(interval_corrections)} 幀")
    print(f"  間隔最大值: {np.max(interval_corrections)} 幀")
    
    # 誤差趨勢分析（檢查是否有單向累積）
    print(f"\n誤差趨勢分析（檢查單向累積）:")
    sample_points = [0, len(rounding_errors)//4, len(rounding_errors)//2, 
                     3*len(rounding_errors)//4, len(rounding_errors)-1]
    
    print(f"  {'位置':>10} | {'誤差':>10} | 趨勢")
    print(f"  {'-'*35}")
    for j, pos in enumerate(sample_points):
        if pos < len(rounding_errors):
            error = rounding_errors[pos]
            print(f"  {pos:10d} | {error:+10.4f} | ", end="")
            
            if j == 0:
                print("(起始)")
            elif j == len(sample_points) - 1:
                if abs(error) < 0.2:
                    print("✅ (穩定)")
                else:
                    print("⚠️ (偏離)")
            else:
                print("")
    
    print("="*70)
    
    return optimized_pairs

# 執行 V5.1 優化
pairs_final = optimize_pairing_with_accumulator(pairs_adaptive, fps, frame_count, target_thermal_fps=8)"""

In [ ]:
"""print(f"\n{'='*70}")
print(f"V5.1 效果驗證（對比 V4 vs V5.1）")
print(f"{'='*70}")

# V4（原始）的 RGB 幀間隔
rgb_intervals_v4 = []
for i in range(1, min(500, len(pairs_adaptive))):
    interval = pairs_adaptive[i]['rgb_frame_idx'] - pairs_adaptive[i-1]['rgb_frame_idx']
    rgb_intervals_v4.append(interval)

# V5.1（優化）的 RGB 幀間隔
rgb_intervals_v5 = []
for i in range(1, min(500, len(pairs_final))):
    interval = pairs_final[i]['rgb_frame_idx'] - pairs_final[i-1]['rgb_frame_idx']
    rgb_intervals_v5.append(interval)

rgb_intervals_v4 = np.array(rgb_intervals_v4)
rgb_intervals_v5 = np.array(rgb_intervals_v5)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. RGB 幀間隔分佈對比
axes[0, 0].hist(rgb_intervals_v4, bins=range(0, max(rgb_intervals_v4)+2), 
                alpha=0.6, label='V4 (Original)', color='#ff6b6b', edgecolor='black')
axes[0, 0].hist(rgb_intervals_v5, bins=range(0, max(rgb_intervals_v5)+2), 
                alpha=0.6, label='V5.1 (Accumulator)', color='#51cf66', edgecolor='black')
axes[0, 0].set_xlabel('RGB Frame Interval')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('RGB Frame Interval Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. RGB 幀間隔時間序列
sample_size = min(200, len(rgb_intervals_v4))
axes[0, 1].plot(range(sample_size), rgb_intervals_v4[:sample_size], 'o-', 
               label='V4', alpha=0.6, markersize=3)
axes[0, 1].plot(range(sample_size), rgb_intervals_v5[:sample_size], 's-', 
               label='V5.1', alpha=0.6, markersize=3)
axes[0, 1].axhline(np.mean(rgb_intervals_v4), color='#ff6b6b', linestyle='--', 
                  label=f'V4 Mean: {np.mean(rgb_intervals_v4):.2f}')
axes[0, 1].axhline(np.mean(rgb_intervals_v5), color='#51cf66', linestyle='--', 
                  label=f'V5.1 Mean: {np.mean(rgb_intervals_v5):.2f}')
axes[0, 1].set_xlabel('Frame Index')
axes[0, 1].set_ylabel('Interval (frames)')
axes[0, 1].set_title(f'RGB Frame Interval Sequence (First {sample_size})')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 四捨五入誤差累積（V5.1 的核心指標）
rounding_errors = [p.get('rounding_error', 0) for p in pairs_final[1:]]
rounding_errors = np.array(rounding_errors[:1000])  # 前 1000 個

axes[1, 0].scatter(range(len(rounding_errors)), rounding_errors, alpha=0.5, s=10, color='#4ecdc4')
axes[1, 0].axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
axes[1, 0].axhline(0.5, color='red', linestyle='--', linewidth=1, alpha=0.5, label='±0.5 boundary')
axes[1, 0].axhline(-0.5, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[1, 0].set_xlabel('Frame Index')
axes[1, 0].set_ylabel('Rounding Error (frames)')
axes[1, 0].set_title('V5.1: Rounding Error Pattern')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. 統計對比表
stats_data = {
    '指標': ['平均間隔', '標準差', '最小', '最大', '穩定性'],
    'V4': [
        f"{np.mean(rgb_intervals_v4):.4f}",
        f"{np.std(rgb_intervals_v4):.4f}",
        f"{np.min(rgb_intervals_v4)}",
        f"{np.max(rgb_intervals_v4)}",
        f"{(1-np.std(rgb_intervals_v4)/np.mean(rgb_intervals_v4))*100:.1f}%"
    ],
    'V5.1': [
        f"{np.mean(rgb_intervals_v5):.4f}",
        f"{np.std(rgb_intervals_v5):.4f}",
        f"{np.min(rgb_intervals_v5)}",
        f"{np.max(rgb_intervals_v5)}",
        f"{(1-np.std(rgb_intervals_v5)/np.mean(rgb_intervals_v5))*100:.1f}%"
    ]
}

axes[1, 1].axis('off')
table = axes[1, 1].table(cellText=[[stats_data['指標'][i], stats_data['V4'][i], stats_data['V5.1'][i]] 
                                   for i in range(len(stats_data['指標']))],
                        colLabels=['Metric', 'V4', 'V5.1'],
                        cellLoc='center',
                        loc='center',
                        bbox=[0, 0, 1, 1])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

plt.suptitle('V4 vs V5.1 配對效果對比', fontsize=14, fontweight='bold')
plt.tight_layout()

save_path = DIAGNOSTIC_DIR / 'v4_vs_v51_comparison.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
plt.close()

print(f"\n對比圖已儲存: {save_path}")

# 輸出結論
print(f"\n結論:")
if np.std(rgb_intervals_v5) < np.std(rgb_intervals_v4):
    improvement = (1 - np.std(rgb_intervals_v5)/np.std(rgb_intervals_v4)) * 100
    print(f"  ✅ V5.1 改善了 {improvement:.1f}%")
    print(f"  RGB 幀間隔變得更加均勻")
    print(f"  Thermal 延遲應該被消除了")
else:
    print(f"  ⚠️ V5.1 沒有改善")
    print(f"  可能需要檢查 RGB FPS 的精確值")

print("="*70)"""

In [ ]:
"""def optimize_with_error_diffusion(pairs: List[Dict], fps: float, frame_count: int, target_thermal_fps: int = 8) -> List[Dict]:
    
    使用誤差擴散（Error Diffusion）優化 RGB 間隔分佈
    
    原理：
    每個 Thermal 幀需要 3.08375 RGB 幀
    3 + 0.08375 = 3.08375
    
    累積 0.08375 × 12 = 1.005 ≈ 1
    所以每 12 個 Thermal 幀，RGB 需要多跳 1 幀
    
    這個方法會把 4-幀間隔均勻分佈到整個序列
    
    
    print(f"\n{'='*70}")
    print(f"最優振盪分佈（誤差擴散）")
    print(f"{'='*70}")
    
    if len(pairs) == 0:
        return pairs
    
    expected_interval = fps / target_thermal_fps  # 3.08375
    
    print(f"\n配置:")
    print(f"  預期 RGB 間隔: {expected_interval:.5f} 幀")
    print(f"  整數部分: {int(expected_interval)} 幀")
    print(f"  小數部分: {expected_interval - int(expected_interval):.5f} 幀")
    
    # 計算周期：多久需要多跳一幀
    fractional = expected_interval - int(expected_interval)
    period = round(1 / fractional)  # 大約是 12
    
    print(f"  週期: 每 {period} 個 Thermal 幀，RGB 多跳 1 幀")
    print(f"  驗證: {period} × {fractional:.5f} = {period * fractional:.5f} ≈ 1 ✓")
    
    optimized_pairs = []
    base_interval = int(expected_interval)  # 3
    
    # 第一幀保持原樣
    first_pair = pairs[0].copy()
    first_pair['interval_applied'] = 0
    first_pair['accumulated_error'] = 0.0
    optimized_pairs.append(first_pair)
    
    accumulated_error = 0.0
    
    for i in range(1, len(pairs)):
        # 累積誤差
        accumulated_error += fractional
        
        # 決定這一步用 3 還是 4
        if accumulated_error >= 1.0:
            interval = base_interval + 1
            accumulated_error -= 1.0
        else:
            interval = base_interval
        
        # 計算新的 RGB 幀索引
        prev_rgb_idx = optimized_pairs[-1]['rgb_frame_idx']
        new_rgb_idx = prev_rgb_idx + interval
        
        # 邊界檢查
        new_rgb_idx = min(new_rgb_idx, frame_count - 1)
        
        pair_opt = pairs[i].copy()
        pair_opt['rgb_frame_idx_original'] = pairs[i]['rgb_frame_idx']
        pair_opt['rgb_frame_idx'] = new_rgb_idx
        pair_opt['interval_applied'] = interval
        pair_opt['accumulated_error'] = accumulated_error
        
        optimized_pairs.append(pair_opt)
    
    print(f"\n✅ 最優化完成")
    print(f"  總對數: {len(optimized_pairs):,}")
    
    # 分析結果
    intervals = [optimized_pairs[i+1]['interval_applied'] for i in range(len(optimized_pairs)-1)]
    intervals = np.array(intervals)
    
    from collections import Counter
    interval_dist = Counter(intervals)
    
    print(f"\nRGB 幀間隔分佈:")
    for interval, count in sorted(interval_dist.items()):
        percentage = count / len(intervals) * 100
        print(f"  間隔 {interval} 幀: {count:5d} 次 ({percentage:5.1f}%)")
    
    # 檢查均勻性
    print(f"\nRGB 幀間隔統計:")
    print(f"  平均: {np.mean(intervals):.5f} 幀")
    print(f"  標準差: {np.std(intervals):.5f} 幀")
    print(f"  最小: {np.min(intervals)} 幀")
    print(f"  最大: {np.max(intervals)} 幀")
    
    # 檢查總消耗
    total_rgb = optimized_pairs[-1]['rgb_frame_idx'] - optimized_pairs[0]['rgb_frame_idx']
    expected_rgb = (len(pairs) - 1) * expected_interval
    
    print(f"\n總 RGB 消耗驗證:")
    print(f"  實際: {total_rgb} 幀")
    print(f"  預期: {expected_rgb:.1f} 幀")
    print(f"  誤差: {total_rgb - expected_rgb:+.1f} 幀")
    
    print("="*70)
    
    return optimized_pairs

# 執行誤差擴散優化
pairs_final = optimize_with_error_diffusion(pairs_adaptive, fps, frame_count, target_thermal_fps=8)"""

---
## Step 7: 生成配對驗證影片

In [ ]:
def create_validation_video(
    pairs: list,
    avi_path: Path,
    camera_start_time: datetime,
    output_path: Path,
    num_samples: int = 500,
    flip_thermal: bool = True,
    flip_rgb: bool = False,
    video_fps: int = 2
):
    """
    生成配對驗證影片（side-by-side）
    """
    print(f"\n{'='*70}")
    print(f"生成配對驗證影片")
    print(f"{'='*70}")
    
    cap = cv2.VideoCapture(str(avi_path))
    
    if not cap.isOpened():
        print(f"❌ 無法開啟影片")
        return False
    
    rgb_fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 取樣策略：均勻取樣整個時間範圍
    step = max(1, len(pairs) // num_samples)
    sample_indices = list(range(0, min(len(pairs), num_samples * step), step))[:num_samples]
    
    print(f"\n配置:")
    print(f"  取樣數量: {len(sample_indices)}")
    print(f"  取樣間隔: 每 {step} 個配對取 1 個")
    print(f"  輸出 FPS: {video_fps}")
    
    # 讀取第一幀獲取尺寸
    first_pair = pairs[sample_indices[0]]
    cap.set(cv2.CAP_PROP_POS_FRAMES, first_pair['rgb_frame_idx'])
    ret, first_rgb = cap.read()
    
    if not ret:
        print(f"❌ 無法讀取第一幀")
        cap.release()
        return False
    
    rgb_height, rgb_width = first_rgb.shape[:2]
    
    # 建立 VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(
        str(output_path),
        fourcc,
        video_fps,
        (rgb_width * 2, rgb_height)
    )
    
    if not out.isOpened():
        print(f"❌ 無法建立輸出影片")
        cap.release()
        return False
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    success_count = 0
    
    for idx, pair_idx in enumerate(tqdm(sample_indices, desc="生成影片")):
        pair = pairs[pair_idx]
        
        try:
            # Thermal
            thermal = pair['thermal']['image'].copy()
            if flip_thermal:
                thermal = cv2.flip(thermal, 1)
            
            thermal_upscaled = cv2.resize(thermal, (rgb_width, rgb_height), 
                                          interpolation=cv2.INTER_CUBIC)
            thermal_colored = cv2.applyColorMap(thermal_upscaled, cv2.COLORMAP_JET)
            
            # RGB
            cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
            ret, rgb = cap.read()
            
            if not ret:
                continue
            
            if flip_rgb:
                rgb = cv2.flip(rgb, 1)
            
            # 計算時間差
            thermal_ts = pair['timestamp']
            rgb_ts = camera_start_time + timedelta(seconds=pair['rgb_frame_idx'] / rgb_fps)
            time_diff = (thermal_ts - rgb_ts).total_seconds()
            
            # Thermal 側標籤
            y_pos = 40
            labels_thermal = [
                f"THERMAL - Pair {pair_idx}",
                f"Time: {thermal_ts.strftime('%H:%M:%S.%f')[:-3]}",
                f"Frame: {pair['frame_number']}",
                f"Progress: {idx+1}/{len(sample_indices)}"
            ]
            
            for label in labels_thermal:
                cv2.putText(thermal_colored, label, (10, y_pos), font, 0.7, (255, 255, 255), 2)
                y_pos += 30
            
            # RGB 側標籤
            y_pos = 40
            text_color = (0, 255, 0) if abs(time_diff) < 0.5 else (0, 165, 255) if abs(time_diff) < 1.0 else (0, 0, 255)
            
            labels_rgb = [
                f"RGB - Frame {pair['rgb_frame_idx']}",
                f"Time: {rgb_ts.strftime('%H:%M:%S.%f')[:-3]}",
                f"Diff: {time_diff:+.3f}s",
                f"Error: {pair['rgb_error_ms']:.2f}ms"
            ]
            
            for label in labels_rgb:
                cv2.putText(rgb, label, (10, y_pos), font, 0.7, text_color, 2)
                y_pos += 30
            
            # 合併並寫入
            combined = np.hstack([thermal_colored, rgb])
            out.write(combined)
            success_count += 1
            
        except Exception as e:
            print(f"\n⚠️ 處理 Pair {pair_idx} 時出錯: {e}")
            continue
    
    cap.release()
    out.release()
    
    print(f"\n✅ 影片生成完成！")
    print(f"  輸出: {output_path}")
    print(f"  成功: {success_count} 幀")
    print(f"  時長: {success_count/video_fps:.1f} 秒")
    
    if output_path.exists():
        file_size = output_path.stat().st_size / (1024 * 1024)
        print(f"  大小: {file_size:.2f} MB")
    
    print("="*70)
    
    return True

# 生成驗證影片
video_path = OUTPUT_DIR / 'pairing_validation_v5_1.mp4'
create_validation_video(
    pairs=pairs_final,
    avi_path=AVI_FILE,
    camera_start_time=camera_start_time,
    output_path=video_path,
    num_samples=5000,
    flip_thermal=FLIP_THERMAL,
    flip_rgb=FLIP_RGB,
    video_fps=10
)

In [ ]:
# ========================================
# Step 8: 建立 LabelMe 專案結構
# ========================================

print(f"\n{'='*70}")
print(f"建立 LabelMe 專案結構")
print(f"{'='*70}")

# 建立子目錄
images_dir = LABELME_DIR / 'images'
annotations_dir = LABELME_DIR / 'annotations'

images_dir.mkdir(exist_ok=True)
annotations_dir.mkdir(exist_ok=True)

print(f"\n建立目錄:")
print(f"  Images: {images_dir}")
print(f"  Annotations: {annotations_dir}")

# 多核匯出函數
def export_labelme_batch(args):
    pairs_batch, avi_path, images_out_dir, flip_rgb, height, width = args
    
    cap = cv2.VideoCapture(str(avi_path))
    exported_count = 0
    
    for pair in pairs_batch:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        if flip_rgb:
            frame = cv2.flip(frame, 1)
        
        pair_id = pair['pair_id'].replace('pair_', '')
        filename = f"frame_{pair_id}.jpg"
        
        # 儲存圖片
        save_path = images_out_dir / filename
        cv2.imwrite(str(save_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
        
        exported_count += 1
    
    cap.release()
    return exported_count

# 分批處理
print(f"\n匯出圖片（使用 {NUM_WORKERS} workers）...")

from concurrent.futures import ProcessPoolExecutor

batch_size = max(100, len(pairs_adaptive) // NUM_WORKERS)
batches = [
    (pairs_adaptive[i:i+batch_size], AVI_FILE, images_dir, FLIP_RGB, height, width)
    for i in range(0, len(pairs_adaptive), batch_size)
]

total_exported = 0

with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(export_labelme_batch, batches),
        total=len(batches),
        desc="匯出圖片"
    ))

total_exported = sum(results)

print(f"✅ 匯出 {total_exported} 張圖片")

# 生成 classes.txt（只要三個類別）
classes_content = """posture_lying
posture_sitting
posture_falling"""

classes_txt_path = LABELME_DIR / 'classes.txt'
with open(classes_txt_path, 'w', encoding='utf-8') as f:
    f.write(classes_content)

print(f"✅ 生成 classes.txt (3 個類別)")

# 生成 pairing_index.json
print(f"\n生成 pairing_index.json...")

index_data = {
    'total_images': len(pairs_adaptive),
    'thermal_rgb_pairs': [
        {
            'image': f"frame_{pair['pair_id'].replace('pair_', '')}.jpg",
            'timestamp': pair['timestamp'].isoformat(),
            'rgb_frame_idx': pair['rgb_frame_idx']
        }
        for pair in pairs_adaptive
    ]
}

index_path = LABELME_DIR / 'pairing_index.json'
with open(index_path, 'w', encoding='utf-8') as f:
    json.dump(index_data, f, indent=2, ensure_ascii=False)

print(f"✅ 生成 pairing_index.json")

# 生成簡短 README
readme_content = f"""# LabelMe 標註專案

## 使用方法

### 1. 用 LabelMe 指令匯出
```bash
cd {LABELME_DIR}
labelme images --labels classes.txt --output annotations"""
readme_path = LABELME_DIR / 'README.md'
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

---
## Step 8: 視覺化配對分析

In [ ]:
# 視覺化配對結果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. RGB 誤差分布
rgb_errors = [p['rgb_error_ms'] for p in pairs_final]
axes[0, 0].hist(rgb_errors, bins=50, edgecolor='black', alpha=0.7, color='#51cf66')
axes[0, 0].axvline(np.mean(rgb_errors), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(rgb_errors):.2f}ms')
axes[0, 0].set_xlabel('RGB Error (ms)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('RGB Time Alignment Error Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. RGB 幀間隔分佈
rgb_indices = [p['rgb_frame_idx'] for p in pairs_final]
intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]

axes[0, 1].hist(intervals, bins=range(0, min(20, max(intervals)+2)), edgecolor='black', alpha=0.7, color='#74c0fc')
expected_interval = FRAME_INTERVAL_MS / (1000 / fps)
axes[0, 1].axvline(expected_interval, color='red', linestyle='--', linewidth=2,
                   label=f'Expected: {expected_interval:.1f} frames')
axes[0, 1].set_xlabel('RGB Frame Interval')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('RGB Frame Interval Distribution')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Thermal 時間間隔
time_diffs = []
for i in range(len(pairs_final) - 1):
    dt = (pairs_final[i+1]['timestamp'] - pairs_final[i]['timestamp']).total_seconds() * 1000
    time_diffs.append(dt)

axes[1, 0].hist(time_diffs, bins=50, edgecolor='black', alpha=0.7, color='#ffa94d')
axes[1, 0].axvline(FRAME_INTERVAL_MS, color='red', linestyle='--', linewidth=2,
                   label=f'Expected: {FRAME_INTERVAL_MS:.1f}ms')
axes[1, 0].set_xlabel('Thermal Time Interval (ms)')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Thermal Frame Time Interval Distribution (Should be constant)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. RGB 幀序列趨勢
sample_size = min(500, len(rgb_indices))
axes[1, 1].plot(range(sample_size), rgb_indices[:sample_size], 'b-', linewidth=1, alpha=0.7)
axes[1, 1].scatter(range(sample_size), rgb_indices[:sample_size], c='blue', s=10, alpha=0.5)
axes[1, 1].set_xlabel('Pair Index')
axes[1, 1].set_ylabel('RGB Frame Index')
axes[1, 1].set_title(f'RGB Frame Sequence (First {sample_size} Pairs)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
save_path = PAIRING_DIR / 'pairing_analysis_v4.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
plt.close()

print(f"\n✅ 視覺化已儲存: {save_path}")

---
## Step 9: 儲存配對結果

In [ ]:
# 保存配對結果
pairs_export = []
for pair in pairs_final:
    pairs_export.append({
        'pair_id': pair['pair_id'],
        'rgb_frame_idx': pair['rgb_frame_idx'],
        'timestamp': pair['timestamp'].isoformat(),
        'frame_number': pair['frame_number'],
        'rgb_error_ms': pair['rgb_error_ms']
    })

# 儲存 JSON
json_path = PAIRING_DIR / 'pairs_mapping_v4.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(pairs_export, f, indent=2, ensure_ascii=False)

# 儲存 CSV
csv_path = PAIRING_DIR / 'pairs_mapping_v4.csv'
df_pairs = pd.DataFrame(pairs_export)
df_pairs.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*70}")
print(f"配對結果已儲存")
print(f"{'='*70}")
print(f"  JSON: {json_path}")
print(f"  CSV: {csv_path}")
print(f"  總配對數: {len(pairs_export):,}")
print("="*70)

---
## Step 10: 驗證修正結果

In [ ]:
# 最終驗證
print(f"\n{'='*70}")
print(f"最終驗證報告")
print(f"{'='*70}")

print(f"\n前 20 個配對:")
for i, pair in enumerate(pairs_final[:20]):
    thermal_time = pair['timestamp'].strftime('%H:%M:%S.%f')[:-3]
    rgb_idx = pair['rgb_frame_idx']
    error = pair['rgb_error_ms']
    
    if i > 0:
        time_diff = (pair['timestamp'] - pairs_final[i-1]['timestamp']).total_seconds() * 1000
        rgb_diff = rgb_idx - pairs_final[i-1]['rgb_frame_idx']
        print(f"{i:2d}. {pair['pair_id']}: Thermal {thermal_time} → RGB 幀 {rgb_idx:5d} "
              f"(誤差 {error:4.1f}ms, 時間間隔 {time_diff:6.1f}ms, RGB間隔 {rgb_diff} 幀)")
    else:
        print(f"{i:2d}. {pair['pair_id']}: Thermal {thermal_time} → RGB 幀 {rgb_idx:5d} (誤差 {error:4.1f}ms)")

# 檢查時間間隔一致性
time_intervals = []
for i in range(len(pairs_final) - 1):
    dt = (pairs_final[i+1]['timestamp'] - pairs_final[i]['timestamp']).total_seconds() * 1000
    time_intervals.append(dt)

print(f"\n\n時間間隔一致性檢查:")
print(f"  平均間隔: {np.mean(time_intervals):.3f}ms")
print(f"  標準差: {np.std(time_intervals):.3f}ms")
print(f"  預期值: {FRAME_INTERVAL_MS:.3f}ms")
print(f"  誤差: {abs(np.mean(time_intervals) - FRAME_INTERVAL_MS):.3f}ms")

if np.std(time_intervals) < 0.1:
    print(f"\n✅ 時間間隔非常穩定！累積誤差問題已解決")
else:
    print(f"\n⚠️ 時間間隔仍有變化，可能需要進一步檢查")

print(f"\n{'='*70}")
print(f"\n📝 後續步驟:")
print(f"1. 檢視驗證影片: {video_path}")
print(f"2. 查看診斷圖: {DIAGNOSTIC_DIR / 'timing_diagnosis_v4.png'}")
print(f"3. 如果同步良好，繼續進行 LabelMe 標註")
print(f"4. 如果仍有問題，檢查 output.txt 的 camera_start_time 是否正確")
print(f"{'='*70}")

In [ ]:
def compare_thermal_rgb_motion(pairs: List[Dict], avi_path: Path, sample_size: int = 50):
    """
    比對 Thermal 和 RGB 的動作內容，看是否有時間偏移
    """
    print(f"\n{'='*70}")
    print(f"比對 Thermal-RGB 運動內容")
    print(f"{'='*70}")
    
    cap = cv2.VideoCapture(str(avi_path))
    
    if not cap.isOpened():
        print("❌ 無法開啟影片")
        return
    
    # 均勻取樣
    step = len(pairs) // sample_size
    sample_indices = range(0, len(pairs), max(1, step))[:sample_size]
    
    print(f"\n取樣 {len(list(sample_indices))} 對進行比對...")
    
    motion_correlations = []
    
    for pair_idx in sample_indices:
        pair = pairs[pair_idx]
        
        # Thermal 前後幀的差異（運動量）
        if pair_idx > 0 and pair_idx < len(pairs) - 1:
            thermal_curr = pair['thermal']['image'].astype(float)
            thermal_prev = pairs[pair_idx - 1]['thermal']['image'].astype(float)
            thermal_next = pairs[pair_idx + 1]['thermal']['image'].astype(float)
            
            # 運動量 = 相鄰幀的平均差異
            thermal_motion = (np.mean(np.abs(thermal_curr - thermal_prev)) + 
                            np.mean(np.abs(thermal_next - thermal_curr))) / 2
            
            # RGB 的相應幀
            rgb_curr_idx = pair['rgb_frame_idx']
            
            # 讀取 RGB 前後幀
            cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, rgb_curr_idx - 1))
            ret1, rgb_prev = cap.read()
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, rgb_curr_idx)
            ret2, rgb_curr = cap.read()
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-1, rgb_curr_idx + 1))
            ret3, rgb_next = cap.read()
            
            if ret1 and ret2 and ret3:
                # 轉灰度
                rgb_prev_gray = cv2.cvtColor(rgb_prev, cv2.COLOR_BGR2GRAY).astype(float)
                rgb_curr_gray = cv2.cvtColor(rgb_curr, cv2.COLOR_BGR2GRAY).astype(float)
                rgb_next_gray = cv2.cvtColor(rgb_next, cv2.COLOR_BGR2GRAY).astype(float)
                
                rgb_motion = (np.mean(np.abs(rgb_curr_gray - rgb_prev_gray)) + 
                            np.mean(np.abs(rgb_next_gray - rgb_curr_gray))) / 2
                
                # 如果兩邊運動量相關，說明配對正確
                if rgb_motion > 0:
                    correlation = thermal_motion / rgb_motion
                    motion_correlations.append(correlation)
    
    if motion_correlations:
        motion_correlations = np.array(motion_correlations)
        
        print(f"\nThermal-RGB 運動相關性:")
        print(f"  平均比率: {np.mean(motion_correlations):.4f}")
        print(f"  標準差: {np.std(motion_correlations):.4f}")
        print(f"  最小: {np.min(motion_correlations):.4f}")
        print(f"  最大: {np.max(motion_correlations):.4f}")
        
        # 視覺化
        plt.figure(figsize=(12, 5))
        plt.hist(motion_correlations, bins=30, edgecolor='black', alpha=0.7, color='#4ecdc4')
        plt.axvline(np.mean(motion_correlations), color='red', linestyle='--', linewidth=2,
                   label=f'Mean: {np.mean(motion_correlations):.4f}')
        plt.xlabel('Thermal-RGB Motion Ratio')
        plt.ylabel('Count')
        plt.title('Motion Correlation Between Thermal and RGB')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        save_path = DIAGNOSTIC_DIR / 'motion_correlation.png'
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        print(f"\n✅ 圖表已儲存: {save_path}")
    
    cap.release()
    print("="*70)

# 執行
compare_thermal_rgb_motion(pairs_adaptive, AVI_FILE, sample_size=50)

In [5]:
# ========================================
# Step 9: Thermal 超解析度（PyTorch + 2080 Super）
# ========================================

import subprocess
import sys
import threading
from pathlib import Path

print(f"\n{'='*70}")
print(f"Thermal 超解析度訓練（PyTorch）")
print(f"{'='*70}")

# 安裝依賴
print("\n安裝依賴套件...")
packages = ['torch', 'torchvision', 'opencv-python', 'numpy', 'pillow']

for pkg in packages:
    try:
        __import__(pkg.replace('-', '_'))
        print(f"✓ {pkg} 已安裝")
    except ImportError:
        print(f"安裝 {pkg}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image

# 檢查 GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"\nGPU 檢查:")
print(f"  設備: {device}")
if torch.cuda.is_available():
    print(f"  GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA 版本: {torch.version.cuda}")
    print(f"  ✓ GPU 可用")

# ========================================
# 準備配對的 Thermal 和 RGB 資料
# ========================================

def prepare_thermal_rgb_pairs(pairs, aligned_dir, output_dir):
    """
    從配對資料準備訓練集
    Thermal 32×24 → RGB 縮小到 32×24，作為目標
    然後訓練 32×24 → 256×192
    """
    
    print(f"\n準備訓練資料...")
    
    # 建立資料目錄
    lr_dir = output_dir / 'lr_thermal'  # 低解析度 Thermal（32×24，輸入）
    hr_dir = output_dir / 'hr_thermal'  # 高解析度目標（256×192，預期輸出）
    
    lr_dir.mkdir(parents=True, exist_ok=True)
    hr_dir.mkdir(parents=True, exist_ok=True)
    
    cap = cv2.VideoCapture(str(AVI_FILE))
    
    print(f"  處理 {min(2000, len(pairs))} 個配對...")
    
    for idx, pair in enumerate(pairs[:min(2000, len(pairs))]):
        # Thermal 低解析度（原始 32×24）
        thermal_lr = pair['thermal']['image']
        lr_path = lr_dir / f"thermal_{idx:05d}.png"
        cv2.imwrite(str(lr_path), thermal_lr)
        
        # RGB 作為高解析度目標
        # 策略：RGB 縮小到 256×192（接近 Thermal 32×24 的 8 倍）
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
        ret, rgb_frame = cap.read()
        
        if ret:
            # RGB 縮小到 256×192（8 倍 32×24）
            rgb_resized = cv2.resize(rgb_frame, (256, 192), interpolation=cv2.INTER_CUBIC)
            # 轉灰階
            rgb_gray = cv2.cvtColor(rgb_resized, cv2.COLOR_BGR2GRAY)
            
            hr_path = hr_dir / f"thermal_{idx:05d}.png"
            cv2.imwrite(str(hr_path), rgb_gray)
    
    cap.release()
    
    print(f"  低解析度 Thermal（32×24）: {len(list(lr_dir.glob('*.png')))} 張")
    print(f"  高解析度目標（256×192）: {len(list(hr_dir.glob('*.png')))} 張")
    
    return lr_dir, hr_dir

# ========================================
# 定義超解析度模型 1: ESPCN（快速）
# ========================================

class ESPCN(nn.Module):
    """
    Efficient Sub-Pixel Convolutional Neural Network
    32×24 → 256×192（8 倍上採樣）
    """
    def __init__(self, upscale_factor=8):
        super().__init__()
        self.upscale_factor = upscale_factor
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 1 * (upscale_factor ** 2), kernel_size=3, padding=1)
        
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
    
    def forward(self, x):
        # x: (batch, 1, 24, 32)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.conv3(x)
        x = self.pixel_shuffle(x)
        # output: (batch, 1, 192, 256)
        return x

# ========================================
# 定義超解析度模型 2: SRResNet（高精度）
# ========================================

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
    
    def forward(self, x):
        residual = x
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x = x + residual
        return x

class SRResNet(nn.Module):
    """
    Super-Resolution ResNet
    高精度，適合對質量要求高的應用
    """
    def __init__(self, num_residual_blocks=16, upscale_factor=8):
        super().__init__()
        self.upscale_factor = upscale_factor
        
        # 初始層
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)
        
        # 殘差塊
        residual_blocks = [ResidualBlock(64) for _ in range(num_residual_blocks)]
        self.residual_blocks = nn.Sequential(*residual_blocks)
        
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(64)
        
        # 上採樣層
        upsample_layers = []
        for _ in range(int(np.log2(upscale_factor))):
            upsample_layers.extend([
                nn.Conv2d(64, 256, kernel_size=3, padding=1),
                nn.PixelShuffle(2)
            ])
        self.upsample = nn.Sequential(*upsample_layers)
        
        # 輸出層
        self.conv3 = nn.Conv2d(64, 1, kernel_size=9, padding=4)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        residual = x
        x = self.residual_blocks(x)
        x = self.bn(self.conv2(x))
        x = x + residual
        x = self.upsample(x)
        x = self.conv3(x)
        return x

# ========================================
# 自訂資料集
# ========================================

class ThermalRGBDataset(Dataset):
    def __init__(self, lr_dir, hr_dir):
        self.lr_files = sorted(lr_dir.glob('*.png'))
        self.hr_files = sorted(hr_dir.glob('*.png'))
        
        assert len(self.lr_files) == len(self.hr_files), f"LR ({len(self.lr_files)}) 和 HR ({len(self.hr_files)}) 數量不符"
        
        print(f"  資料集: {len(self.lr_files)} 對圖片")
    
    def __len__(self):
        return len(self.lr_files)
    
    def __getitem__(self, idx):
        # 載入低解析度 Thermal（32×24）
        lr_img = cv2.imread(str(self.lr_files[idx]), cv2.IMREAD_GRAYSCALE)
        if lr_img is None:
            lr_img = np.zeros((24, 32), dtype=np.uint8)
        lr_img = torch.FloatTensor(lr_img).unsqueeze(0) / 255.0  # (1, 24, 32)
        
        # 載入高解析度目標（256×192，來自 RGB 縮小版）
        hr_img = cv2.imread(str(self.hr_files[idx]), cv2.IMREAD_GRAYSCALE)
        if hr_img is None:
            hr_img = np.zeros((192, 256), dtype=np.uint8)
        hr_img = torch.FloatTensor(hr_img).unsqueeze(0) / 255.0  # (1, 192, 256)
        
        return lr_img, hr_img

# ========================================
# 訓練函式
# ========================================

def train_super_resolution(model_name, model, train_loader, device, epochs=50):
    """
    訓練超解析度模型
    """
    print(f"\n訓練 {model_name}...")
    
    criterion = nn.L1Loss()  # L1 Loss 更適合 SR
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for lr_img, hr_img in train_loader:
            lr_img = lr_img.to(device)
            hr_img = hr_img.to(device)
            
            # 前向傳播
            sr_img = model(lr_img)
            
            # 計算損失
            loss = criterion(sr_img, hr_img)
            
            # 反向傳播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        scheduler.step()
        
        avg_loss = total_loss / len(train_loader)
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.6f}")
    
    print(f"✅ {model_name} 訓練完成")
    
    return model

# ========================================
# 主程式：並行訓練兩個模型
# ========================================

def train_espcn(lr_dir, hr_dir, output_dir, device):
    print(f"\n{'='*70}")
    print(f"【模型 1】ESPCN 訓練...")
    print(f"{'='*70}")
    
    try:
        dataset = ThermalRGBDataset(lr_dir, hr_dir)
        train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
        
        model = ESPCN(upscale_factor=8)
        
        model = train_super_resolution('ESPCN', model, train_loader, device, epochs=30)
        
        # 保存模型
        model_path = output_dir / 'espcn_sr.pth'
        torch.save(model.state_dict(), str(model_path))
        print(f"  模型: {model_path}")
        
    except Exception as e:
        print(f"❌ ESPCN 訓練失敗: {e}")
        import traceback
        traceback.print_exc()

def train_srresnet(lr_dir, hr_dir, output_dir, device):
    print(f"\n{'='*70}")
    print(f"【模型 2】SRResNet 訓練...")
    print(f"{'='*70}")
    
    try:
        dataset = ThermalRGBDataset(lr_dir, hr_dir)
        train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)
        
        model = SRResNet(num_residual_blocks=16, upscale_factor=8)
        
        model = train_super_resolution('SRResNet', model, train_loader, device, epochs=50)
        
        # 保存模型
        model_path = output_dir / 'srresnet_sr.pth'
        torch.save(model.state_dict(), str(model_path))
        print(f"  模型: {model_path}")
        
    except Exception as e:
        print(f"❌ SRResNet 訓練失敗: {e}")
        import traceback
        traceback.print_exc()

# ========================================
# 執行
# ========================================

"""print(f"\n準備訓練資料...")

output_base = LABELME_DIR.parent / 'sr_models'
output_base.mkdir(exist_ok=True)

lr_dir, hr_dir = prepare_thermal_rgb_pairs(pairs_adaptive, ALIGNED_DIR, output_base)

print(f"\n開始並行訓練超解析度模型...")
print(f"使用 GPU: {device}")

threads = []

# 線程 1: ESPCN
t1 = threading.Thread(
    target=train_espcn,
    args=(lr_dir, hr_dir, output_base, device),
    daemon=False
)
threads.append(('ESPCN', t1))

# 線程 2: SRResNet
t2 = threading.Thread(
    target=train_srresnet,
    args=(lr_dir, hr_dir, output_base, device),
    daemon=False
)
threads.append(('SRResNet', t2))

# 啟動
for name, t in threads:
    print(f"\n🚀 啟動 {name} 訓練...")
    t.start()

# 等待完成
print(f"\n⏳ 等待訓練完成...")
for name, t in threads:
    t.join()
    print(f"✓ {name} 完成")

print(f"\n{'='*70}")
print(f"✅ 超解析度模型訓練完成！")
print(f"{'='*70}")
print(f"\n模型位置:")
print(f"  ESPCN: {output_base}/espcn_sr.pth")
print(f"  SRResNet: {output_base}/srresnet_sr.pth")
print(f"\n用途:")
print(f"  - 32×24 Thermal → 256×192 (8 倍超解析度)")
print(f"  - ESPCN: 快速、輕量")
print(f"  - SRResNet: 高精度、細節豐富")
print("="*70)"""


Thermal 超解析度訓練（PyTorch）

安裝依賴套件...
✓ torch 已安裝
✓ torchvision 已安裝
安裝 opencv-python...
✓ numpy 已安裝
安裝 pillow...

GPU 檢查:
  設備: cuda:0
  GPU 名稱: NVIDIA GeForce RTX 2080 SUPER
  CUDA 版本: 12.8
  ✓ GPU 可用


'print(f"\n準備訓練資料...")\n\noutput_base = LABELME_DIR.parent / \'sr_models\'\noutput_base.mkdir(exist_ok=True)\n\nlr_dir, hr_dir = prepare_thermal_rgb_pairs(pairs_adaptive, ALIGNED_DIR, output_base)\n\nprint(f"\n開始並行訓練超解析度模型...")\nprint(f"使用 GPU: {device}")\n\nthreads = []\n\n# 線程 1: ESPCN\nt1 = threading.Thread(\n    target=train_espcn,\n    args=(lr_dir, hr_dir, output_base, device),\n    daemon=False\n)\nthreads.append((\'ESPCN\', t1))\n\n# 線程 2: SRResNet\nt2 = threading.Thread(\n    target=train_srresnet,\n    args=(lr_dir, hr_dir, output_base, device),\n    daemon=False\n)\nthreads.append((\'SRResNet\', t2))\n\n# 啟動\nfor name, t in threads:\n    print(f"\n🚀 啟動 {name} 訓練...")\n    t.start()\n\n# 等待完成\nprint(f"\n⏳ 等待訓練完成...")\nfor name, t in threads:\n    t.join()\n    print(f"✓ {name} 完成")\n\nprint(f"\n{\'=\'*70}")\nprint(f"✅ 超解析度模型訓練完成！")\nprint(f"{\'=\'*70}")\nprint(f"\n模型位置:")\nprint(f"  ESPCN: {output_base}/espcn_sr.pth")\nprint(f"  SRResNet: {output_base}/srresnet_sr.pth")\nprin

In [ ]:
# ========================================
# Step 10: 超解析度推理對照（GPU 加速版 + 自訂範圍）
# ========================================

print(f"\n{'='*70}")
print(f"超解析度推理對照（GPU 加速）")
print(f"{'='*70}")

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')

def inference_and_visualize(model, lr_dir, output_dir, model_name, device, start_idx=100, end_idx=110, sample_every=1):
    """
    用訓練好的模型做推理，生成對照圖（GPU 加速）
    
    參數:
    - start_idx: 開始索引
    - end_idx: 結束索引
    - sample_every: 每 N 張取 1 張（可加速）
    """
    
    print(f"\n推理 {model_name}...")
    print(f"  範圍: 第 {start_idx} ~ {end_idx} 張")
    print(f"  取樣間隔: 每 {sample_every} 張取 1 張")
    
    # 取指定範圍的圖片
    all_lr_files = sorted(lr_dir.glob('*.png'))
    lr_files = all_lr_files[start_idx:end_idx:sample_every]
    
    print(f"  實際使用: {len(lr_files)} 張圖片")
    print(f"  總圖片數: {len(all_lr_files)}")
    
    if not lr_files:
        print(f"❌ 沒有找到圖片")
        return
    
    model.eval()
    model.to(device)
    
    results = []
    inference_times = []
    
    with torch.no_grad():
        for idx, lr_path in enumerate(lr_files):
            # 載入低解析度 Thermal
            lr_img = cv2.imread(str(lr_path), cv2.IMREAD_GRAYSCALE)
            
            if lr_img is None:
                print(f"  ⚠️ 無法載入: {lr_path.name}")
                continue
            
            print(f"  [{idx+1}/{len(lr_files)}] 處理: {lr_path.name} (shape: {lr_img.shape})", end="")
            
            # 轉為 tensor，並移到 GPU
            lr_img_tensor = torch.FloatTensor(lr_img).unsqueeze(0).unsqueeze(0) / 255.0
            lr_img_tensor = lr_img_tensor.to(device)
            
            # 推理計時
            import time
            start_time = time.time()
            
            with torch.cuda.device(device):
                sr_img_tensor = model(lr_img_tensor)
            
            inference_time = time.time() - start_time
            inference_times.append(inference_time)
            
            print(f" → {inference_time*1000:.2f}ms", end="")
            
            # 結果移回 CPU 並轉為 numpy
            sr_img = sr_img_tensor.squeeze().detach().cpu().numpy() * 255.0
            sr_img = np.clip(sr_img, 0, 255).astype(np.uint8)
            
            print(f" → 輸出 shape: {sr_img.shape}")
            
            # 簡單的雙線性插值作為對照
            simple_upscale = cv2.resize(lr_img, (256, 192), interpolation=cv2.INTER_LINEAR)
            
            results.append({
                'lr': lr_img,
                'sr': sr_img,
                'simple': simple_upscale,
                'filename': lr_path.name
            })
    
    print(f"\n✅ {len(results)} 張圖片推理完成（在 GPU 上）")
    print(f"\n推理效能統計:")
    print(f"  平均推理時間: {np.mean(inference_times)*1000:.2f}ms")
    print(f"  最小推理時間: {np.min(inference_times)*1000:.2f}ms")
    print(f"  最大推理時間: {np.max(inference_times)*1000:.2f}ms")
    
    # 生成對照圖
    num_samples = len(results)
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4 * num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for idx, result in enumerate(results):
        # 低解析度（32×24）
        axes[idx, 0].imshow(result['lr'], cmap='hot')
        axes[idx, 0].set_title(f'Low Res (32×24)\n{result["filename"]}', fontsize=9)
        axes[idx, 0].axis('off')
        
        # 模型輸出（256×192）
        axes[idx, 1].imshow(result['sr'], cmap='hot')
        axes[idx, 1].set_title(f'{model_name}\n(256×192)', fontsize=9)
        axes[idx, 1].axis('off')
        
        # 簡單插值對照（256×192）
        axes[idx, 2].imshow(result['simple'], cmap='hot')
        axes[idx, 2].set_title('Simple Upscale\n(256×192)', fontsize=9)
        axes[idx, 2].axis('off')
    
    plt.tight_layout()
    output_path = output_base / f'{model_name}_inference_[{start_idx}-{end_idx}].png'
    plt.savefig(str(output_path), dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"\n✅ 對照圖已儲存: {output_path}")
    
    return output_path

# ========================================
# 載入訓練好的模型並推理
# ========================================

print(f"\n檢查 GPU 狀態:")
print(f"  設備: {device}")
print(f"  CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU 名稱: {torch.cuda.get_device_name(device)}")
    print(f"  GPU 記憶體: {torch.cuda.get_device_properties(device).total_memory / 1e9:.2f} GB")

output_base = LABELME_DIR.parent / 'sr_models'
lr_dir = output_base / 'lr_thermal'

# 檢查總圖片數
total_images = len(list(lr_dir.glob('*.png')))
print(f"\n總圖片數: {total_images}")

# ========================================
# 自訂參數區域（你可以改這裡）
# ========================================

# 方案 1: 看前 10 張
START_IDX_1 = 0
END_IDX_1 = 10

# 方案 2: 看中間 10 張
START_IDX_2 = 100
END_IDX_2 = 110

# 方案 3: 看後面 10 張
START_IDX_3 = total_images - 10
END_IDX_3 = total_images

print(f"\n推理範圍選項:")
print(f"  方案 1 (前 10 張): [{START_IDX_1}:{END_IDX_1}]")
print(f"  方案 2 (中間 10 張): [{START_IDX_2}:{END_IDX_2}]")
print(f"  方案 3 (後 10 張): [{START_IDX_3}:{END_IDX_3}]")

print(f"\n載入模型...")

# ESPCN 推理
espcn_path = output_base / 'espcn_sr.pth'
if espcn_path.exists():
    print(f"\n【推理】ESPCN...")
    espcn_model = ESPCN(upscale_factor=8)
    espcn_model.load_state_dict(torch.load(str(espcn_path), map_location=device))
    espcn_model.to(device)
    
    # 推理方案 2（可改成 1 或 3）
    espcn_output = inference_and_visualize(
        espcn_model, lr_dir, output_base, 'ESPCN', device,
        start_idx=START_IDX_2,
        end_idx=END_IDX_2,
        sample_every=1
    )
else:
    print(f"❌ ESPCN 模型不存在: {espcn_path}")

print()

# SRResNet 推理
srresnet_path = output_base / 'srresnet_sr.pth'
if srresnet_path.exists():
    print(f"\n【推理】SRResNet...")
    srresnet_model = SRResNet(num_residual_blocks=16, upscale_factor=8)
    srresnet_model.load_state_dict(torch.load(str(srresnet_path), map_location=device))
    srresnet_model.to(device)
    
    # 推理方案 2（可改成 1 或 3）
    srresnet_output = inference_and_visualize(
        srresnet_model, lr_dir, output_base, 'SRResNet', device,
        start_idx=START_IDX_2,
        end_idx=END_IDX_2,
        sample_every=1
    )
else:
    print(f"❌ SRResNet 模型不存在: {srresnet_path}")

print(f"\n{'='*70}")
print(f"✅ 推理對照完成（已使用 GPU 加速）")
print(f"{'='*70}")
print(f"\n對照圖位置:")
print(f"  ESPCN: {output_base}/ESPCN_inference_[{START_IDX_2}-{END_IDX_2}].png")
print(f"  SRResNet: {output_base}/SRResNet_inference_[{START_IDX_2}-{END_IDX_2}].png")
print(f"\n對照圖說明:")
print(f"  左欄: 原始低解析度 Thermal (32×24)")
print(f"  中欄: 模型超解析度輸出 (256×192)")
print(f"  右欄: 簡單雙線性插值對照 (256×192)")
print(f"\n評估重點:")
print(f"  ✓ 模型輸出是否比插值版本更清晰細節豐富？")
print(f"  ✓ 是否保留了 Thermal 的邊界和特徵？")
print(f"  ✓ 是否有過度平滑或偽影？")
print("="*70)

# 清空 GPU 記憶體
torch.cuda.empty_cache()
print(f"\n✓ GPU 記憶體已清空")


超解析度推理對照（GPU 加速）

檢查 GPU 狀態:
  設備: cuda:0
  CUDA 可用: True
  GPU 名稱: NVIDIA GeForce RTX 2080 SUPER
  GPU 記憶體: 8.16 GB

總圖片數: 2000

推理範圍選項:
  方案 1 (前 10 張): [0:10]
  方案 2 (中間 10 張): [100:110]
  方案 3 (後 10 張): [1990:2000]

載入模型...

【推理】ESPCN...

推理 ESPCN...
  範圍: 第 100 ~ 110 張
  取樣間隔: 每 1 張取 1 張
  實際使用: 10 張圖片
  總圖片數: 2000
  [1/10] 處理: thermal_00100.png (shape: (24, 32)) → 0.34ms → 輸出 shape: (192, 256)
  [2/10] 處理: thermal_00101.png (shape: (24, 32)) → 0.30ms → 輸出 shape: (192, 256)
  [3/10] 處理: thermal_00102.png (shape: (24, 32)) → 0.27ms → 輸出 shape: (192, 256)
  [4/10] 處理: thermal_00103.png (shape: (24, 32)) → 0.29ms → 輸出 shape: (192, 256)
  [5/10] 處理: thermal_00104.png (shape: (24, 32)) → 0.33ms → 輸出 shape: (192, 256)
  [6/10] 處理: thermal_00105.png (shape: (24, 32)) → 0.27ms → 輸出 shape: (192, 256)
  [7/10] 處理: thermal_00106.png (shape: (24, 32)) → 0.26ms → 輸出 shape: (192, 256)
  [8/10] 處理: thermal_00107.png (shape: (24, 32)) → 0.26ms → 輸出 shape: (192, 256)
  [9/10] 處理: thermal_00108.pn